In [1]:
# === ALL-GROUP ML (coeffs + disc only, NO invariants) — 60/40 split ===
# Sage/Jupyter compatible

import os, ast, json, hashlib

import numpy as np
import pandas as pd
from builtins import int as pyint, float as pyfloat  # avoid Sage Integer/Real issues

from sklearn.model_selection import (
    train_test_split, StratifiedKFold, GroupKFold
)

# StratifiedGroupKFold may not exist in all sklearn versions
try:
    from sklearn.model_selection import StratifiedGroupKFold
    HAS_SGF = True
except Exception:
    HAS_SGF = False

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import check_random_state

from joblib import dump

# ---------------- Config ----------------
CSV_PATH = "/Users/jurimezini/Library/CloudStorage/Dropbox/Sage_Galois7/AIMS-Galois-7/AIMS-7.csv"
OUT_DIR  = os.path.join(os.path.dirname(CSV_PATH), "models_all_noinv_60_40")
os.makedirs(OUT_DIR, exist_ok=True)

RANDOM   = pyint(42)

# For this run we want the FULL dataset (no quick subsampling)
QUICK    = False
N_QUICK  = pyint(120000)   # only used if QUICK=True for testing

# Feature toggles
USE_COEFFS = True
USE_DISC   = True      # disc_sign + disc_log10abs
USE_JINV   = False     # <--- NO invariants here

# Split controls
TEST_FRACTION     = pyfloat(0.40)  # 60/40 split
USE_GROUPED_SPLIT = False          # family-aware split off for now

# Canonical label space for ALL groups
CANON_ALL = ["S7", "A7", "PSL(3,2)", "C7 ⋊ C3", "D7", "C7 ⋊ C6", "C7"]

# Normalize some group names if needed
NORMALIZE_GROUP = {
    "L(3,2": "PSL(3,2)",
    "L(3,2)": "PSL(3,2)",
    "7:2": "D7",
    "7:6": "C7 ⋊ C6",
}

# Map 7T* codes to math notation if we ever use galois_label
MAP_7T = {
    "7T7": "S7",
    "7T6": "A7",
    "7T5": "PSL(3,2)",
    "7T3": "C7 ⋊ C3",
    "7T2": "D7",
    "7T4": "C7 ⋊ C6",
    "7T1": "C7",
}

# ---------------- Helpers ----------------
def signed_log10(X):
    """
    Elementwise sgn(x)*log10(1+|x|); kept for completeness if you later add j-invariants.
    """
    X = np.asarray(X, dtype=np.float64)
    return np.sign(X) * np.log10(1.0 + np.abs(X))

def class_weight_samples(y_vec, power=0.5):
    """
    Per-sample weights ~ (median_count / class_count)^power to soften imbalance.
    """
    power = float(power)
    vals, counts = np.unique(y_vec, return_counts=True)
    med = np.median(counts)
    wmap = {c: (med / cnt) ** power for c, cnt in zip(vals, counts)}
    return np.array([wmap[c] for c in y_vec], dtype=np.float64), wmap

def parse_coeffs_tuple(cell):
    """
    Accept '(...)', '[...]', tuple/list/ndarray; return list[int] of length 8.
    For AIMS-7.csv, 'coeffs' is like "[1, 1, -1, 0, 1, -1, -1, 1]".
    """
    if isinstance(cell, (list, tuple, np.ndarray)):
        v = list(cell)
    elif isinstance(cell, str):
        v = ast.literal_eval(cell)
    else:
        v = cell
    if v is None or len(v) != 8:
        raise ValueError(f"Expected 8 coefficients, got {v}")
    return [int(x) for x in v]

def stable_tuple_hash(row8):
    """
    Deterministic hash for an 8-int coefficient tuple (for grouping).
    """
    b = (",".join(map(str, row8))).encode("utf-8")
    return int(hashlib.sha1(b).hexdigest()[:12], 16)  # 48-bit int

# ---------------- Read CSV (robust) ----------------
df = pd.read_csv(CSV_PATH, low_memory=False)
df.columns = [str(c).strip() for c in df.columns]
print("COLUMNS in file:")
print(df.columns.tolist())
print("Rows total:", len(df))

# Prefer explicit a0..a7; expand tuple only if needed
coeff_cols = [f"a{i}" for i in range(8)]
have_all_coeffs = all(c in df.columns for c in coeff_cols)

if not have_all_coeffs:
    tuple_col = None
    for alt in ["coeffs_tuple_(a0..a7)", "coeffs_tuple", "coeffs"]:
        if alt in df.columns:
            tuple_col = alt
            break
    if tuple_col is None:
        raise ValueError("Need coefficients: either a0..a7 or a tuple column must be present.")

    df[tuple_col] = df[tuple_col].apply(parse_coeffs_tuple)
    coeff_mat = np.vstack(df[tuple_col].to_numpy()).astype(np.int64)
    df_coeffs = pd.DataFrame(coeff_mat, columns=coeff_cols, copy=False)
    df = pd.concat(
        [df.drop(columns=[tuple_col]).reset_index(drop=True),
         df_coeffs.reset_index(drop=True)],
        axis=1
    )

# Drop leftover textual 'coeffs' column to avoid confusion
if "coeffs" in df.columns and "coeffs" not in coeff_cols:
    try:
        df = df.drop(columns=["coeffs"])
    except Exception:
        pass

# ---- Add/repair discriminant-derived features if only raw 'disc' exists ----
if USE_DISC:
    if "disc_sign" not in df.columns and "disc" in df.columns:
        df["disc_sign"] = np.sign(pd.to_numeric(df["disc"], errors="coerce")).astype("float64")
    if "disc_log10abs" not in df.columns and "disc" in df.columns:
        df["disc_log10abs"] = np.log10(
            1.0 + np.abs(pd.to_numeric(df["disc"], errors="coerce"))
        )

# ---- Force numeric types where appropriate ----
numeric_cols = [c for c in
                ["disc", "disc_sign", "disc_log10abs",
                 "j0", "j1", "j2", "j3", "j4"] + coeff_cols
                if c in df.columns]

for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---- Resolve/normalize labels robustly ----
label_col = None
for cand in ["group", "math_galois_notation", "galois_label"]:
    if cand in df.columns:
        label_col = cand
        break

if label_col is None:
    raise ValueError(f"No label column found; have columns: {list(df.columns)}")

if label_col == "galois_label":
    # Map 7T* codes to math notation
    y_raw = df["galois_label"].astype(str).str.strip().map(MAP_7T)
else:
    # For AIMS-7.csv this will be 'math_galois_notation'
    y_raw = df[label_col].astype(str).str.strip()

y_raw = y_raw.replace(NORMALIZE_GROUP)

# Keep only canonical 7 classes (drop anything weird/typo)
mask = y_raw.isin(CANON_ALL)
df   = df.loc[mask].reset_index(drop=True)
y    = y_raw.loc[mask].to_numpy(dtype=object)

print("Detected label column:", label_col)
print("Rows (canonical classes only):", len(df))
print("Class counts:", {c: int((y == c).sum()) for c in CANON_ALL})

# ---- Optional quick subset (OFF for this full-data run) ----
if QUICK and len(y) > N_QUICK:
    rng = check_random_state(0)
    idx = rng.choice(len(y), size=N_QUICK, replace=False)
    df, y = df.iloc[idx].reset_index(drop=True), y[idx]
    print(f"[QUICK] Using subset of {len(y)} rows")

# ---------------- Feature assembly ----------------
logj_cols  = [c for c in ["j0", "j1", "j2", "j3", "j4"] if USE_JINV and c in df.columns]
disc_cols  = [c for c in ["disc_sign", "disc_log10abs"] if USE_DISC and c in df.columns]
use_coeffs = [c for c in coeff_cols if USE_COEFFS and c in df.columns]

transformers = []

# (logj_cols will be empty since USE_JINV=False, but kept for future)
if logj_cols:
    transformers.append(
        ("logj",
         Pipeline([
             ("imp", SimpleImputer(strategy="median")),
             ("log", FunctionTransformer(signed_log10, feature_names_out="one-to-one")),
         ]),
         logj_cols)
    )

if disc_cols:
    transformers.append(
        ("disc",
         SimpleImputer(strategy="median"),
         disc_cols)
    )

if use_coeffs:
    transformers.append(
        ("coeffs",
         Pipeline([
             ("imp", SimpleImputer(strategy="constant", fill_value=pyint(0))),
         ]),
         use_coeffs)
    )

pre = ColumnTransformer(
    transformers=transformers,
    remainder="drop",
    verbose_feature_names_out=False
)

clf = HistGradientBoostingClassifier(
    loss="log_loss",
    learning_rate=0.08,
    max_iter=500,
    max_leaf_nodes=31,
    min_samples_leaf=30,
    l2_regularization=1e-4,
    early_stopping=True,
    n_iter_no_change=20,
    random_state=RANDOM,
)

pipe = Pipeline([("pre", pre), ("clf", clf)])

# ---------------- Build groups for optional leakage-resistant split ----------------
if USE_GROUPED_SPLIT:
    coeff_mat = df[[f"a{i}" for i in range(8)]].to_numpy(dtype=np.int64)
    g_hash = np.array([stable_tuple_hash(row) for row in coeff_mat], dtype=np.int64)
    height_bin = (
        pd.to_numeric(df.get("height", 0), errors="coerce")
        .fillna(0).astype(np.int64).to_numpy()
    )
    groups = (height_bin.astype(np.int64) * 10**6 + (g_hash % 10**6)).astype(np.int64)
else:
    groups = None

# ---------------- External 60/40 hold-out ----------------
if USE_GROUPED_SPLIT:
    from sklearn.model_selection import GroupShuffleSplit
    gss = GroupShuffleSplit(n_splits=1, test_size=float(TEST_FRACTION),
                            random_state=RANDOM)
    tr_idx, te_idx = next(gss.split(df, y, groups=groups))
    X_tr, X_te = df.iloc[tr_idx], df.iloc[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]
    g_tr, g_te = groups[tr_idx], groups[te_idx]
else:
    X_tr, X_te, y_tr, y_te = train_test_split(
        df, y,
        test_size=float(TEST_FRACTION),
        random_state=RANDOM,
        stratify=y
    )
    g_tr = g_te = None

print(f"Train size: {len(y_tr)};  Test size: {len(y_te)} (test fraction {TEST_FRACTION:.2f})")

# ---------------- 5-fold CV on TRAIN ----------------
cv_scores = []

if USE_GROUPED_SPLIT and HAS_SGF:
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=RANDOM)
    splitter = cv.split(X_tr, y_tr, groups=g_tr)
elif USE_GROUPED_SPLIT:
    cv = GroupKFold(n_splits=5)
    splitter = cv.split(X_tr, groups=g_tr)
else:
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM)
    splitter = cv.split(X_tr, y_tr)

for fold, (tr_idx, va_idx) in enumerate(splitter, 1):
    y_tr_fold = y_tr[tr_idx] if isinstance(y_tr, np.ndarray) else np.asarray(y_tr)[tr_idx]
    w_fold, _ = class_weight_samples(y_tr_fold, power=0.5)
    pipe.fit(X_tr.iloc[tr_idx], y_tr_fold, clf__sample_weight=w_fold)
    pred_va = pipe.predict(X_tr.iloc[va_idx])
    ba = balanced_accuracy_score(y_tr[va_idx], pred_va)
    cv_scores.append(ba)
    print(f"Fold {fold}: balanced accuracy = {ba:.4f}")

print(f"[CV on train] Balanced accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# ---------------- Final fit on ALL TRAIN; evaluate on TEST ----------------
w_tr_all, wmap = class_weight_samples(y_tr, power=0.5)
print("Weight map (train):", {k: round(v, 3) for k, v in wmap.items()})

pipe.fit(X_tr, y_tr, clf__sample_weight=w_tr_all)

pred_te = pipe.predict(X_te)
print(f"\n[Test {int(TEST_FRACTION*100)}%] Balanced accuracy: "
      f"{balanced_accuracy_score(y_te, pred_te):.4f}\n")

print("[Test] Classification report")
print(classification_report(y_te, pred_te, labels=CANON_ALL,
                            digits=4, zero_division=0))

print("\n[Test] Confusion matrix (rows=true, cols=pred):")
cm = pd.DataFrame(
    confusion_matrix(y_te, pred_te, labels=CANON_ALL),
    index=CANON_ALL,
    columns=CANON_ALL
)
print(cm)

# ---------------- Save model + config ----------------
model_path = os.path.join(OUT_DIR, "allgroups_noinv_60_40_full.joblib")
dump(pipe, model_path)
print("Saved model to:", model_path)

config = {
    "label_space": [str(c) for c in CANON_ALL],
    "columns_in": [str(c) for c in df.columns],
    "feature_blocks": {
        "coeffs": [str(c) for c in use_coeffs],
        "disc":   [str(c) for c in disc_cols],
        "j_invariants": [str(c) for c in logj_cols],
    },
    "quick_mode": bool(QUICK),
    "test_fraction": float(TEST_FRACTION),
    "grouped_split": bool(USE_GROUPED_SPLIT),
}
with open(os.path.join(OUT_DIR, "config.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

try:
    feats_out = pipe.named_steps["pre"].get_feature_names_out()
    with open(os.path.join(OUT_DIR, "features_out.txt"), "w", encoding="utf-8") as f:
        for nm in feats_out:
            f.write(f"{nm}\n")
except Exception:
    pass

print("Config and feature names saved in:", OUT_DIR)

# ---------------- One-off prediction helper ----------------
def predict_one(coeffs8=None,
                disc_sign=None,
                disc_log10abs=None):
    """
    Predict a Galois group for a single septic using the trained model.
    Inputs:
      - coeffs8: list/tuple of 8 integers [a0,...,a7]
      - disc_sign: sign of discriminant (-1,0,1) or None
      - disc_log10abs: log10(1+|disc|) or None
    """
    row = {}

    if USE_COEFFS:
        if coeffs8 is None or len(coeffs8) != 8:
            raise ValueError("coeffs8 must be length-8 when USE_COEFFS=True")
        for i, v in enumerate(coeffs8):
            row[f"a{i}"] = int(v)

    if USE_DISC:
        row["disc_sign"]     = None if disc_sign is None else int(disc_sign)
        row["disc_log10abs"] = None if disc_log10abs is None else float(disc_log10abs)

    # Build a one-row DataFrame with exactly the same input columns used in training
    cols_in = pipe.named_steps["pre"].feature_names_in_
    X1 = pd.DataFrame([row], columns=cols_in)
    return pipe.predict(X1)[0]


COLUMNS in file:
['coeffs', 'height', 'disc', 'galois_label', 'math_galois_notation']
Rows total: 1686353
Detected label column: math_galois_notation
Rows (canonical classes only): 1686353
Class counts: {'S7': 1559957, 'A7': 56997, 'PSL(3,2)': 40977, 'C7 ⋊ C3': 24457, 'D7': 2163, 'C7 ⋊ C6': 1550, 'C7': 252}
Train size: 1011811;  Test size: 674542 (test fraction 0.40)
Fold 1: balanced accuracy = 0.7996
Fold 2: balanced accuracy = 0.8237
Fold 3: balanced accuracy = 0.8002
Fold 4: balanced accuracy = 0.8128
Fold 5: balanced accuracy = 0.8154
[CV on train] Balanced accuracy: 0.8103 ± 0.0093
Weight map (train): {'A7': 0.655, 'C7': 9.858, 'C7 ⋊ C3': 1.0, 'C7 ⋊ C6': 3.972, 'D7': 3.362, 'PSL(3,2)': 0.773, 'S7': 0.125}

[Test 40%] Balanced accuracy: 0.8236

[Test] Classification report
              precision    recall  f1-score   support

          S7     0.9992    0.9802    0.9896    623983
          A7     0.6999    0.8744    0.7775     22799
    PSL(3,2)     0.5401    0.5631    0.5513     1

In [2]:
# === ALL-GROUP ML (coefficients only, NO disc, NO invariants) — 60/40 split ===
# Sage/Jupyter compatible

import os, ast, json, hashlib

import numpy as np
import pandas as pd
from builtins import int as pyint, float as pyfloat  # avoid Sage Integer/Real issues

from sklearn.model_selection import (
    train_test_split, StratifiedKFold, GroupKFold
)

# StratifiedGroupKFold may not exist in all sklearn versions
try:
    from sklearn.model_selection import StratifiedGroupKFold
    HAS_SGF = True
except Exception:
    HAS_SGF = False

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import check_random_state

from joblib import dump

# ---------------- Config ----------------
CSV_PATH = "/Users/jurimezini/Library/CloudStorage/Dropbox/Sage_Galois7/AIMS-Galois-7/AIMS-7.csv"
OUT_DIR  = os.path.join(os.path.dirname(CSV_PATH), "models_all_coeffs_only_60_40")
os.makedirs(OUT_DIR, exist_ok=True)

RANDOM   = pyint(42)

# For this run we want the FULL dataset (no quick subsampling)
QUICK    = False
N_QUICK  = pyint(120000)   # only used if QUICK=True for testing

# Feature toggles
USE_COEFFS = True
USE_DISC   = False     # <--- NO discriminant features
USE_JINV   = False     # <--- NO invariants

# Split controls
TEST_FRACTION     = pyfloat(0.40)  # 60/40 split
USE_GROUPED_SPLIT = False          # family-aware split off for now

# Canonical label space for ALL groups
CANON_ALL = ["S7", "A7", "PSL(3,2)", "C7 ⋊ C3", "D7", "C7 ⋊ C6", "C7"]

# Normalize some group names if needed
NORMALIZE_GROUP = {
    "L(3,2": "PSL(3,2)",
    "L(3,2)": "PSL(3,2)",
    "7:2": "D7",
    "7:6": "C7 ⋊ C6",
}

# Map 7T* codes to math notation if we ever use galois_label
MAP_7T = {
    "7T7": "S7",
    "7T6": "A7",
    "7T5": "PSL(3,2)",
    "7T3": "C7 ⋊ C3",
    "7T2": "D7",
    "7T4": "C7 ⋊ C6",
    "7T1": "C7",
}

# ---------------- Helpers ----------------
def signed_log10(X):
    X = np.asarray(X, dtype=np.float64)
    return np.sign(X) * np.log10(1.0 + np.abs(X))

def class_weight_samples(y_vec, power=0.5):
    power = float(power)
    vals, counts = np.unique(y_vec, return_counts=True)
    med = np.median(counts)
    wmap = {c: (med / cnt) ** power for c, cnt in zip(vals, counts)}
    return np.array([wmap[c] for c in y_vec], dtype=np.float64), wmap

def parse_coeffs_tuple(cell):
    """
    Accept '(...)', '[...]', tuple/list/ndarray; return list[int] of length 8.
    For AIMS-7.csv, 'coeffs' is like "[1, 1, -1, 0, 1, -1, -1, 1]".
    """
    if isinstance(cell, (list, tuple, np.ndarray)):
        v = list(cell)
    elif isinstance(cell, str):
        v = ast.literal_eval(cell)
    else:
        v = cell
    if v is None or len(v) != 8:
        raise ValueError(f"Expected 8 coefficients, got {v}")
    return [int(x) for x in v]

def stable_tuple_hash(row8):
    b = (",".join(map(str, row8))).encode("utf-8")
    return int(hashlib.sha1(b).hexdigest()[:12], 16)  # 48-bit int

# ---------------- Read CSV (robust) ----------------
df = pd.read_csv(CSV_PATH, low_memory=False)
df.columns = [str(c).strip() for c in df.columns]
print("COLUMNS in file:")
print(df.columns.tolist())
print("Rows total:", len(df))

# Prefer explicit a0..a7; expand tuple only if needed
coeff_cols = [f"a{i}" for i in range(8)]
have_all_coeffs = all(c in df.columns for c in coeff_cols)

if not have_all_coeffs:
    tuple_col = None
    for alt in ["coeffs_tuple_(a0..a7)", "coeffs_tuple", "coeffs"]:
        if alt in df.columns:
            tuple_col = alt
            break
    if tuple_col is None:
        raise ValueError("Need coefficients: either a0..a7 or a tuple column must be present.")

    df[tuple_col] = df[tuple_col].apply(parse_coeffs_tuple)
    coeff_mat = np.vstack(df[tuple_col].to_numpy()).astype(np.int64)
    df_coeffs = pd.DataFrame(coeff_mat, columns=coeff_cols, copy=False)
    df = pd.concat(
        [df.drop(columns=[tuple_col]).reset_index(drop=True),
         df_coeffs.reset_index(drop=True)],
        axis=1
    )

# Drop leftover textual 'coeffs' column to avoid confusion
if "coeffs" in df.columns and "coeffs" not in coeff_cols:
    try:
        df = df.drop(columns=["coeffs"])
    except Exception:
        pass

# ---- Force numeric types where appropriate ----
numeric_cols = [c for c in
                ["disc", "disc_sign", "disc_log10abs",
                 "j0", "j1", "j2", "j3", "j4"] + coeff_cols
                if c in df.columns]

for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---- Resolve/normalize labels robustly ----
label_col = None
for cand in ["group", "math_galois_notation", "galois_label"]:
    if cand in df.columns:
        label_col = cand
        break

if label_col is None:
    raise ValueError(f"No label column found; have columns: {list(df.columns)}")

if label_col == "galois_label":
    y_raw = df["galois_label"].astype(str).str.strip().map(MAP_7T)
else:
    y_raw = df[label_col].astype(str).str.strip()

y_raw = y_raw.replace(NORMALIZE_GROUP)

# Keep only canonical 7 classes (drop anything weird/typo)
mask = y_raw.isin(CANON_ALL)
df   = df.loc[mask].reset_index(drop=True)
y    = y_raw.loc[mask].to_numpy(dtype=object)

print("Detected label column:", label_col)
print("Rows (canonical classes only):", len(df))
print("Class counts:", {c: int((y == c).sum()) for c in CANON_ALL})

# ---- Optional quick subset (OFF for this full-data run) ----
if QUICK and len(y) > N_QUICK:
    rng = check_random_state(0)
    idx = rng.choice(len(y), size=N_QUICK, replace=False)
    df, y = df.iloc[idx].reset_index(drop=True), y[idx]
    print(f"[QUICK] Using subset of {len(y)} rows")

# ---------------- Feature assembly ----------------
logj_cols  = [c for c in ["j0", "j1", "j2", "j3", "j4"] if USE_JINV and c in df.columns]
disc_cols  = []  # USE_DISC=False, so empty
use_coeffs = [c for c in coeff_cols if USE_COEFFS and c in df.columns]

transformers = []

if disc_cols:
    transformers.append(
        ("disc",
         SimpleImputer(strategy="median"),
         disc_cols)
    )

if use_coeffs:
    transformers.append(
        ("coeffs",
         Pipeline([
             ("imp", SimpleImputer(strategy="constant", fill_value=pyint(0))),
         ]),
         use_coeffs)
    )

pre = ColumnTransformer(
    transformers=transformers,
    remainder="drop",
    verbose_feature_names_out=False
)

clf = HistGradientBoostingClassifier(
    loss="log_loss",
    learning_rate=0.08,
    max_iter=500,
    max_leaf_nodes=31,
    min_samples_leaf=30,
    l2_regularization=1e-4,
    early_stopping=True,
    n_iter_no_change=20,
    random_state=RANDOM,
)

pipe = Pipeline([("pre", pre), ("clf", clf)])

# ---------------- Build groups for optional leakage-resistant split ----------------
if USE_GROUPED_SPLIT:
    coeff_mat = df[[f"a{i}" for i in range(8)]].to_numpy(dtype=np.int64)
    g_hash = np.array([stable_tuple_hash(row) for row in coeff_mat], dtype=np.int64)
    height_bin = (
        pd.to_numeric(df.get("height", 0), errors="coerce")
        .fillna(0).astype(np.int64).to_numpy()
    )
    groups = (height_bin.astype(np.int64) * 10**6 + (g_hash % 10**6)).astype(np.int64)
else:
    groups = None

# ---------------- External 60/40 hold-out ----------------
if USE_GROUPED_SPLIT:
    from sklearn.model_selection import GroupShuffleSplit
    gss = GroupShuffleSplit(n_splits=1, test_size=float(TEST_FRACTION),
                            random_state=RANDOM)
    tr_idx, te_idx = next(gss.split(df, y, groups=groups))
    X_tr, X_te = df.iloc[tr_idx], df.iloc[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]
    g_tr, g_te = groups[tr_idx], groups[te_idx]
else:
    X_tr, X_te, y_tr, y_te = train_test_split(
        df, y,
        test_size=float(TEST_FRACTION),
        random_state=RANDOM,
        stratify=y
    )
    g_tr = g_te = None

print(f"Train size: {len(y_tr)};  Test size: {len(y_te)} (test fraction {TEST_FRACTION:.2f})")

# ---------------- 5-fold CV on TRAIN ----------------
cv_scores = []

if USE_GROUPED_SPLIT and HAS_SGF:
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=RANDOM)
    splitter = cv.split(X_tr, y_tr, groups=g_tr)
elif USE_GROUPED_SPLIT:
    cv = GroupKFold(n_splits=5)
    splitter = cv.split(X_tr, groups=g_tr)
else:
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM)
    splitter = cv.split(X_tr, y_tr)

for fold, (tr_idx, va_idx) in enumerate(splitter, 1):
    y_tr_fold = y_tr[tr_idx] if isinstance(y_tr, np.ndarray) else np.asarray(y_tr)[tr_idx]
    w_fold, _ = class_weight_samples(y_tr_fold, power=0.5)
    pipe.fit(X_tr.iloc[tr_idx], y_tr_fold, clf__sample_weight=w_fold)
    pred_va = pipe.predict(X_tr.iloc[va_idx])
    ba = balanced_accuracy_score(y_tr[va_idx], pred_va)
    cv_scores.append(ba)
    print(f"Fold {fold}: balanced accuracy = {ba:.4f}")

print(f"[CV on train] Balanced accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# ---------------- Final fit on ALL TRAIN; evaluate on TEST ----------------
w_tr_all, wmap = class_weight_samples(y_tr, power=0.5)
print("Weight map (train):", {k: round(v, 3) for k, v in wmap.items()})

pipe.fit(X_tr, y_tr, clf__sample_weight=w_tr_all)

pred_te = pipe.predict(X_te)
print(f"\n[Test {int(TEST_FRACTION*100)}%] Balanced accuracy: "
      f"{balanced_accuracy_score(y_te, pred_te):.4f}\n")

print("[Test] Classification report")
print(classification_report(y_te, pred_te, labels=CANON_ALL,
                            digits=4, zero_division=0))

print("\n[Test] Confusion matrix (rows=true, cols=pred):")
cm = pd.DataFrame(
    confusion_matrix(y_te, pred_te, labels=CANON_ALL),
    index=CANON_ALL,
    columns=CANON_ALL
)
print(cm)

# ---------------- Save model + config ----------------
model_path = os.path.join(OUT_DIR, "allgroups_coeffs_only_60_40_full.joblib")
dump(pipe, model_path)
print("Saved model to:", model_path)

config = {
    "label_space": [str(c) for c in CANON_ALL],
    "columns_in": [str(c) for c in df.columns],
    "feature_blocks": {
        "coeffs": [str(c) for c in use_coeffs],
        "disc":   [],          # no disc features
        "j_invariants": [],    # no invariants
    },
    "quick_mode": bool(QUICK),
    "test_fraction": float(TEST_FRACTION),
    "grouped_split": bool(USE_GROUPED_SPLIT),
}
with open(os.path.join(OUT_DIR, "config.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

try:
    feats_out = pipe.named_steps["pre"].get_feature_names_out()
    with open(os.path.join(OUT_DIR, "features_out.txt"), "w", encoding="utf-8") as f:
        for nm in feats_out:
            f.write(f"{nm}\n")
except Exception:
    pass

print("Config and feature names saved in:", OUT_DIR)

# ---------------- One-off prediction helper ----------------
def predict_one(coeffs8=None):
    """
    Predict a Galois group for a single septic using the trained model.
    Input:
      - coeffs8: list/tuple of 8 integers [a0,...,a7]
    """
    row = {}

    if USE_COEFFS:
        if coeffs8 is None or len(coeffs8) != 8:
            raise ValueError("coeffs8 must be length-8 when USE_COEFFS=True")
        for i, v in enumerate(coeffs8):
            row[f"a{i}"] = int(v)

    cols_in = pipe.named_steps["pre"].feature_names_in_
    X1 = pd.DataFrame([row], columns=cols_in)
    return pipe.predict(X1)[0]


COLUMNS in file:
['coeffs', 'height', 'disc', 'galois_label', 'math_galois_notation']
Rows total: 1686353
Detected label column: math_galois_notation
Rows (canonical classes only): 1686353
Class counts: {'S7': 1559957, 'A7': 56997, 'PSL(3,2)': 40977, 'C7 ⋊ C3': 24457, 'D7': 2163, 'C7 ⋊ C6': 1550, 'C7': 252}
Train size: 1011811;  Test size: 674542 (test fraction 0.40)
Fold 1: balanced accuracy = 0.7491
Fold 2: balanced accuracy = 0.7596
Fold 3: balanced accuracy = 0.7357
Fold 4: balanced accuracy = 0.7625
Fold 5: balanced accuracy = 0.7559
[CV on train] Balanced accuracy: 0.7526 ± 0.0095
Weight map (train): {'A7': 0.655, 'C7': 9.858, 'C7 ⋊ C3': 1.0, 'C7 ⋊ C6': 3.972, 'D7': 3.362, 'PSL(3,2)': 0.773, 'S7': 0.125}

[Test 40%] Balanced accuracy: 0.7726

[Test] Classification report
              precision    recall  f1-score   support

          S7     0.9934    0.9653    0.9792    623983
          A7     0.5314    0.8270    0.6470     22799
    PSL(3,2)     0.4344    0.4552    0.4446     1